In [ ]:
import matplotlib.pyplot as plt

from turtle import forward
import numpy as np
import pandas as pd
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

from torch.utils.data import Dataset, DataLoader

import time
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import time
from scipy.fft import fft

from pathlib import Path
import torchaudio
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms  

import torch.nn.functional as F





from sklearn.metrics import accuracy_score, confusion_matrix


def acc_m(test_dataset, model):
    device = next(model.parameters()).device
    y_test = []
    y_test_hat = []
    model.eval()
    
    for inputs, labels in test_dataset:
        # Add batch dimension like in training
        inputs = inputs.unsqueeze(0)  # [freq, time] -> [1, freq, time]
        
        with torch.no_grad():
            outputs = model(inputs.to(device))
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
        
        y_test.append(labels)  # labels is already an int
        y_test_hat.extend(preds)

    acc = accuracy_score(y_test, y_test_hat)
    print(f'Accuracy: {acc*100:.2f} %')
    cm = confusion_matrix(y_test, y_test_hat)
    print(cm)

In [ ]:
class AudioFolder(Dataset):
    def __init__(self, root, transform=None, target_transform=None,
                 extensions=(".wav", ".flac", ".mp3"), allowed_classes=None):
        self.root = Path(root)
        self.transform = transform
        self.target_transform = target_transform
        self.extensions = extensions
        self.allowed_classes = allowed_classes

        # crawl sub-dirs and record (path, class_idx)
        all_classes = sorted({p.parent.name for p in self.root.rglob("*") if p.suffix in extensions})
        
        # Filter classes if allowed_classes is provided
        if allowed_classes is not None:
            self.classes = [c for c in all_classes if c in allowed_classes]
        else:
            self.classes = all_classes
            
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}
        
        # Only include samples from allowed classes
        self.samples = [(p, self.class_to_idx[p.parent.name])
                        for p in self.root.rglob("*") 
                        if p.suffix in extensions and p.parent.name in self.classes]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        waveform, sr = torchaudio.load(path, backend="ffmpeg")

        if self.transform:
            waveform = self.transform(waveform)
        if self.target_transform:
            label = self.target_transform(label)

        return waveform[0], label

# TODO - it seems it introduces randomness here
transform_wav = lambda x: torchaudio.functional.spectrogram(
        x,
        pad=0,
        win_length = 255,
        window=torch.hann_window(255),  # Hann window
        n_fft=255,
        normalized=False,
        center=False ,
        hop_length=128,                  # overlap = n_fft - hop_length
        power=None                       # None -> keep complex, 2.0 -> magnitude^2
    )

ALLOWED_CLASSES = ['normal', 'murmur', 'extrahls', 'artifact']
dataset_wav = AudioFolder('../data/hb/', transform_wav, allowed_classes=ALLOWED_CLASSES)
dataset_wav_train, dataset_wav_test = train_test_split(dataset_wav, train_size=0.9)


class AudioClassifier(nn.Module):
    def __init__(self, n_classes, *args, **kwargs):
        super().__init__(*args, **kwargs)
        def build_Seq():
            return nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            # nn.BatchNorm2d(16),
            nn.Conv2d(32, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            )
        self.real = build_Seq()

        self.imag = build_Seq()

        in_size = 24000 * 2
        self.fc = nn.Sequential(
            nn.Linear(in_size, in_size // 8),
            nn.Linear(in_size // 8, in_size // 32),
            nn.Linear(in_size // 32, n_classes)
        )

    def forward(self, x):
        real_input = x.real.unsqueeze(1)
        img_input = x.imag.unsqueeze(1)

        out_l = self.real(real_input)   
        out_r = self.imag(img_input)        

        out_l = F.adaptive_avg_pool2d(out_l, (30, 100))  # Fixed reasonable size
        out_r = F.adaptive_avg_pool2d(out_r, (30, 100))  # [batch, 8, 30, 100]

        out_l = out_l.flatten(1)  # [batch, 24000]
        out_r = out_r.flatten(1)  # [batch, 24000]
        out = torch.cat([out_l, out_r], dim=1)  # [batch, 2*channels*freq*time]
        return self.fc(out)
    

class AudioClassifier2(nn.Module):
    def __init__(self, n_classes, *args, **kwargs):
        super().__init__(*args, **kwargs)
        def build_Seq():
            return nn.Sequential(
                nn.Conv2d(1, 6, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2,2),
                nn.BatchNorm2d(6),
                nn.Conv2d(6, 16, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2,2),
                nn.BatchNorm2d(16),
                nn.AdaptiveAvgPool2d((16, 32)),  # [batch, 8, 30, 100]
                nn.Flatten(1)
            )
        self.real = build_Seq()

        self.imag = build_Seq()

        in_size = 16384
        self.fc = nn.Sequential(
            nn.Linear(in_size, 128),
            nn.Linear(128, 64),
            nn.Linear(64, n_classes)
        )

    def forward(self, x):
        # print(x.real.shape)
        real_input = x.real.unsqueeze(1)
        img_input = x.imag.unsqueeze(1)
        # real_input = x.real
        # img_input = x.imag     

        out_l = self.real(real_input)   
        out_r = self.imag(img_input)        

        out = torch.cat([out_l, out_r], dim=1)  # [batch, 2*channels*freq*time]
        # print(out.shape)
        return self.fc(out)
    
    
ac = AudioClassifier(len(dataset_wav.classes))
res_t = ac(dataset_wav_test[0][0].unsqueeze(0))
res_t.shape

In [ ]:
import random
data_path = '../data/hb/'

for i in range(10):
    unl = 'unlabelled'
    unlab = Path(data_path,unl)
    random_unlabel = random.choice(list(unlab.glob('*')))
    waveform, sr = torchaudio.load(random_unlabel, backend="ffmpeg")
    transformed = fft(waveform.reshape(-1).numpy())
    print(transformed.shape)
# STFT: windowing + overlap + FFT all in one
    spec = torchaudio.functional.spectrogram(
        waveform,
        pad=0,
        win_length = 255,
        window=torch.hann_window(255),  # Hann window
        n_fft=255,
        normalized=False,
        center=False ,
        hop_length=128,                  # overlap = n_fft - hop_length
        power=None                       # None -> keep complex, 2.0 -> magnitude^2
    )
    print(spec.shape)    

In [ ]:
sm = dataset_wav_test.pop

In [ ]:
# sm = sm()
sm[0].shape

In [ ]:

train_dataset = dataset_wav_train
test_dataset = dataset_wav_test


loss = nn.CrossEntropyLoss()
device = 'cuda'
ac = ac.to(device)
optimiser = torch.optim.Adam(ac.parameters(), lr=0.0001)

NUM_EPOCHS = 20
losses = []

for epoch in range(NUM_EPOCHS):
    for i, (inputs, labels) in enumerate(train_dataset):
        optimiser.zero_grad()
        
        # Add batch dimension manually
        inputs = inputs.unsqueeze(0)  # [1, freq, time]
        labels = torch.tensor([labels])  # [1]
        
        y_pred = ac(inputs.to(device))
        loss_v = loss(y_pred, labels.to(device))
        
        loss_v.backward()
        losses.append(loss_v.item())
        optimiser.step()
        
        if i % 50 == 0:
            print(f'Epoch {epoch}/{NUM_EPOCHS}, Sample {i}, Loss: {loss_v.item():.4f}')
    
    print(f'Completed Epoch {epoch+1}')
    acc_m(test_dataset, ac)

In [ ]:

batch_size = 1 # should've convert that into spectogram
trainloader = DataLoader(dataset_wav_train, batch_size=batch_size, shuffle=True, num_workers=8)
testloader = DataLoader(dataset_wav_test, batch_size=batch_size, shuffle=False,num_workers=8)


from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

loss = nn.CrossEntropyLoss()
device = 'cuda'
ac = ac.to(device)
optimiser = torch.optim.Adam(ac.parameters())

NUM_EPOCHS = 100
losses = []

for epoch in range(NUM_EPOCHS):
    for i, (inputs, labels) in enumerate(trainloader, 0):
        optimiser.zero_grad()

        
        y_pred = ac(inputs.to(device))
        loss_v = loss(y_pred, labels.to(device))
        
        loss_v.backward()
        losses.append(loss_v.item())
        optimiser.step()
        
        if i % 50 == 0:
            print(f'Epoch {epoch}/{NUM_EPOCHS}, Sample {i}, Loss: {loss_v.item():.4f}')
    
    print(f'Completed Epoch {epoch+1}')
    ac.eval()
    with torch.no_grad() :            
        y_real = []
        y_preds = []
        for i, (inputs, labels) in enumerate(testloader, 0):
            y_real.append(labels)
            y_preds.append(torch.argmax(ac(inputs.to(device)), dim=1).cpu().numpy())
        y_real = np.asarray(y_real).flatten()
        y_preds = np.asarray(y_preds).flatten()
        cm = confusion_matrix(y_real, y_preds)

        print("\nPer-class accuracy:")
        for i, cls in enumerate(dataset_wav.classes):
            class_acc = cm[i,i] / cm[i].sum() if cm[i].sum() > 0 else 0
            print(f"{cls}: {class_acc*100:.1f}% ({cm[i,i]}/{cm[i].sum()})")

        # Total accuracy
        total_acc = cm.diagonal().sum() / cm.sum()
        print(f"Total Accuracy: {total_acc*100:.2f}%")            
    ac.train()

In [ ]:
def print_class_distribution(data_subset, original_dataset, name="Dataset"):
    class_counts = {}
    for _, label in data_subset:
        class_counts[label] = class_counts.get(label, 0) + 1
    
    print(f"\n{name} - Class Distribution:")
    print(f"{'Class':<15} {'Count':<10} {'Percentage':<10}")
    print("-" * 35)
    total = len(data_subset)
    for class_name in original_dataset.classes:
        class_idx = original_dataset.class_to_idx[class_name]
        count = class_counts.get(class_idx, 0)
        pct = (count / total * 100) if total > 0 else 0
        print(f"{class_name:<15} {count:<10} {pct:>6.1f}%")
    print(f"{'Total':<15} {total:<10}")

# Use original dataset for class info
print_class_distribution(dataset_wav_train, dataset_wav, "Training Set")
print_class_distribution(dataset_wav_test, dataset_wav, "Test Set")

In [ ]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset

# Get all labels for stratification
all_labels = [label for _, label in dataset_wav]

# Cross-validation loop
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_idx, test_idx) in enumerate(skf.split(range(len(dataset_wav)), all_labels)):
    print(f"\n{'='*50}")
    print(f"FOLD {fold + 1}/5")
    print(f"{'='*50}")
    
    # Create subsets for this fold
    train_subset = Subset(dataset_wav, train_idx)
    test_subset = Subset(dataset_wav, test_idx)
    
    # Create dataloaders
    trainloader = DataLoader(train_subset, batch_size=1, shuffle=True, num_workers=8)
    testloader = DataLoader(test_subset, batch_size=1, shuffle=False, num_workers=8)
    
    # Fresh model for each fold
    ac = AudioClassifier2(len(dataset_wav.classes)).to('cuda')
    optimiser = torch.optim.Adam(ac.parameters(), lr=0.0001, weight_decay=1e-4)
    loss = nn.CrossEntropyLoss()
    
    # Training loop
    for epoch in range(20):  # Fewer epochs per fold
        ac.train()
        for i, (inputs, labels) in enumerate(trainloader):
            optimiser.zero_grad()
            y_pred = ac(inputs.to('cuda'))
            loss_v = loss(y_pred, labels.to('cuda'))
            loss_v.backward()
            optimiser.step()
        
        if epoch % 5 == 0:
            print(f"Fold {fold+1}, Epoch {epoch}, Loss: {loss_v.item():.4f}")
    
    # Evaluate this fold
    ac.eval()
    y_real, y_preds = [], []
    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = ac(inputs.to('cuda'))
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            y_real.extend(labels.numpy())
            y_preds.extend(preds)
    
    acc = (np.array(y_real) == np.array(y_preds)).mean()
    fold_accuracies.append(acc)
    print(f"Fold {fold+1} Accuracy: {acc*100:.2f}%")

print(f"\nAverage Accuracy: {np.mean(fold_accuracies)*100:.2f}% (+/- {np.std(fold_accuracies)*100:.2f}%)")

In [ ]:
from collections import Counter


Counter(y_real)